# How to deal with complex/large Documents

In the previous notebook, we developed a solution for various types of files and data formats commonly found in organizations, and this covers 90% of the use cases. However, you will find that there are issues when dealing with questions that require answers from complex files. The complexity of these files arises from their length and the way information is distributed within them. Large documents are always a challenge for Search Engines.

One example of such complex files is Technical Specification Guides or Product Manuals, which can span hundreds of pages and contain information in the form of images, tables, forms, and more. Books are also complex due to their length and the presence of images or tables.

These files are typically in PDF format. To better handle these PDFs, we need a smarter parsing method that treats each document as a special source and processes them page by page. The objective is to obtain more accurate and faster answers from our system. Fortunately, there are usually not many of these types of documents in an organization, allowing us to make exceptions and treat them differently.

If your use case is just PDFs, for example, you can just use [PyPDF library](https://pypi.org/project/pypdf/) or [Azure AI Document Intelligence SDK (former Form Recognizer)](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-3.0.0), vectorize using OpenAI API and push the content to a vector-based index. And this is problably the simplest and fastest way to go.  However if your use case entails connecting to a datalake, or Sharepoint libraries or any other document data source with thousands of documents with multiple file types and that can change dynamically, then you would want to use the Ingestion and Document Cracking and AI-Enrichment capabilities of Azure Search engine, Notebooks 1-3, and avoid a lot of painful custom code. 


In [11]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

import html

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import  DocumentModelAdministrationClient
from tabulate import tabulate

from common.utils import parse_pdf, read_pdf_files, text_to_base64
from common.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_PROMPT_TEMPLATE
from common.utils import (
    get_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    num_tokens_from_string
)


from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))
    
os.makedirs("data/books/",exist_ok=True)
    

BLOB_CONTAINER_NAME = "auflastung "#"books"
BASE_CONTAINER_URL = "https://storagegenaiasinfo.blob.core.windows.net/" + BLOB_CONTAINER_NAME + "/" #"https://demodatasetsp.blob.core.windows.net/" + BLOB_CONTAINER_NAME + "/"
LOCAL_FOLDER = "./data/books"
MODEL = "gpt-35-turbo" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k

os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [3]:
embedder = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 

## 1 - Manual Document Cracking with Push to Vector-based Index

Within our demo storage account, we have a container named `books`, which holds 5 books of different lengths, languages, and complexities. Let's create a `cogsrch-index-books-vector` and load it with the pages of all these books.

We begin by downloading these books to our local machine:

### What to use: pyPDF or AI Documment Intelligence API (Form Recognizer)?

In `utils.py` there is a **parse_pdf()** function. This utility function can parse local files using PyPDF library and can also parse local or from_url PDFs files using Azure AI Document Intelligence (Former Form Recognizer).

If `form_recognizer=False`, the function will parse the PDF using the python pyPDF library, which 75% of the time does a good job.<br>

Setting `form_recognizer=True`, is the best (and slower) parsing method using AI Documment Intelligence API (former known as Form Recognizer). You can specify the prebuilt model to use, the default is `model="prebuilt-document"`. However, if you have a complex document with tables, charts and figures , you can try
`model="prebuilt-layout"`, and it will capture all of the nuances of each page (it takes longer of course).

**Note: Many PDFs are scanned images. For example, any signed contract that was scanned and saved as PDF will NOT be parsed by pyPDF. Only AI Documment Intelligence API will work.**

In [5]:
#list all pdf files in all folders:
def list_files_in_folder(folder_path):
    pdf_file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".pdf"):
                file_path = os.path.join(root, file)
                pdf_file_paths.append(file_path)
    return pdf_file_paths


file_paths_list = list_files_in_folder(LOCAL_FOLDER)
#for file_path in file_paths_list:
    #print(file_path)

In [6]:
print(len(file_paths_list))

1


In [7]:
def table_to_html(table):
    table_html = "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

In [8]:
def parse_custom_pdf(file, form_recognizer=False, formrecognizer_endpoint=None, formrecognizerkey=None, model="prebuilt-document", from_url=False, verbose=False):

    credential = AzureKeyCredential(os.environ["FORM_RECOGNIZER_KEY"])
    form_recognizer_client = DocumentAnalysisClient(endpoint=os.environ["FORM_RECOGNIZER_ENDPOINT"], credential=credential)
            
    docUrl = "https://storagegenaiasinfo.blob.core.windows.net/auflastung/Daimler Ladungssicherung 9.5.pdf"
    #poller = document_analysis_client.begin_analyze_document_from_url(model_id=model_id, document_url=docUrl)
    #poller = form_recognizer_client.begin_analyze_document_from_url(model_id="documentnumber2", document_url= file)
    with open(file, "rb") as filename:
        poller = form_recognizer_client.begin_analyze_document(model, document = filename)
    
    form_recognizer_results = poller.result()

    #set title
    titel=""
    for i, paragraph in enumerate(form_recognizer_results.paragraphs):
        if(paragraph.role=="title"):        
            titel=titel + " " +paragraph.content
            #print(titel)



    offset = 0
    page_map = []


    # Display key value pairs
    for idx, document in enumerate(form_recognizer_results.documents):
        for name, field in document.fields.items():
            field_value = field.value if field.value else field.content
            #if field.value_type != 'list':
            #    doc_numb=field.value

    for page_num, page in enumerate(form_recognizer_results.pages):
        tables_on_page = [table for table in form_recognizer_results.tables if table.bounding_regions[0].page_number == page_num + 1]

        # mark all positions of the table spans in the page
        page_offset = page.spans[0].offset
        page_length = page.spans[0].length
        table_chars = [-1]*page_length
        for table_id, table in enumerate(tables_on_page):
            for span in table.spans:
                # replace all table spans with "table_id" in table_chars array
                for i in range(span.length):
                    idx = span.offset - page_offset + i
                    if idx >=0 and idx < page_length:
                        table_chars[idx] = table_id

        # build page text by replacing charcters in table spans with table html
        page_text = ""
        added_tables = set()
        for idx, table_id in enumerate(table_chars):
            if table_id == -1:
                page_text += form_recognizer_results.content[page_offset + idx]
            elif not table_id in added_tables:
                page_text += table_to_html(tables_on_page[table_id])
                added_tables.add(table_id)



        #build all sectionheadings of page
        paragraph_on_page = [paragraph for paragraph in form_recognizer_results.paragraphs if paragraph.bounding_regions[0].page_number  == page_num + 1]

        sectionHeading=""
            #set sectionHeadings
        for i, paragraph in enumerate(paragraph_on_page):
            if(paragraph.role=="sectionHeading"):
                sectionHeading += " "+ paragraph.content

        page_text += " "
        page_map.append((page_num, offset, page_text, sectionHeading))
        offset += len(page_text)

        #print(page_map)
    return page_map

In [9]:
book_pages_map = dict()
for book in file_paths_list:
#for book in books:
    print("Extracting Text from",book,"...")
    
    # Capture the start time
    start_time = time.time()
    
    # Parse the PDF
    #book_path = LOCAL_FOLDER+book
    book_map = parse_custom_pdf(file=book, form_recognizer=True, model="prebuilt-layout", verbose=True) #switch here between pyPDF and form recognizer
    book_pages_map[book]= book_map
    
    # Capture the end time and Calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Parsing took: {elapsed_time:.6f} seconds")
    print(f"{book} contained {len(book_map)} pages\n")

Extracting Text from ./data/books/DaimlerLadungssicherung95.pdf ...
Parsing took: 13.120049 seconds
./data/books/DaimlerLadungssicherung95.pdf contained 38 pages



As we can see above, all books were parsed except `Pere_Riche_Pere_Pauvre.pdf` (this book is "Rich Dad, Poor Dad" written in French), why? Well, as we mentioned above, this book was scanned, so each page is an image and with a very unique font. We need a good PDF parser with good OCR capabilities in order to extract the content of this PDF. 
Let's try to parse this book again, but this time using Azure Document Intelligence API (former Form Recognizer)

As demonstrated above, Azure Document Intelligence proves to be superior to pyPDF. **For production scenarios, we strongly recommend using Azure Document Intelligence consistently**. When doing so, it's important to make a wise choice between the available models, such as "prebuilt-document," "prebuilt-layout," or others. You can find more information on model selection [HERE](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/choose-model-feature?view=doc-intel-3.0.0).


## Create Vector-based index


Now that we have the content of the book's chunks (each page of each book) in the dictionary `book_pages_map`, let's create the Vector-based index in our Azure Search Engine where this content is going to land

In [10]:
book_index_name = "custom-auflastung-use-headings" #choose index name, only small letters

In [13]:
### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

In [14]:
index_payload = {
    "name": book_index_name,
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunkVector","type": "Collection(Edm.Single)","searchable": "true","retrievable": "true","dimensions": 1536,"vectorSearchConfiguration": "vectorConfig"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "page_num","type": "Edm.Int32","searchable": "false","retrievable": "true"},
        {"name": "sectionheading","type": "Edm.String","searchable": "true","retrievable": "true"},
        
    ],
    "vectorSearch": {
        "algorithmConfigurations": [
            {
                "name": "vectorConfig",
                "kind": "hnsw"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": [
                        {
                            "fieldName": "sectionheading"
                        }
                    ]
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [20]:
# Uncomment to debug errors
# r.text

'{"error":{"code":"InvalidName","message":"Index name must only contain lowercase letters, digits or dashes, cannot start or end with dashes and is limited to 128 characters.","details":[{"code":"InvalidIndexName","message":"Index name must only contain lowercase letters, digits or dashes, cannot start or end with dashes and is limited to 128 characters."}]}}'

## Upload the Document chunks and its vectors to the Vector-Based Index

The following code will iterate over each chunk of each book and use the Azure Search Rest API upload method to insert each document with its corresponding vector (using OpenAI embedding model) to the index.

In [15]:
for bookname,bookmap in book_pages_map.items():
    #print("Uploading chunks from",bookname)
    for page in tqdm(bookmap):
        #print(page)
        try:
            page_num = page[0] + 1
            content = page[2]
            book_url = BASE_CONTAINER_URL + bookname
            sectionheading=page[3]
            upload_payload = {
                "value": [
                    {
                        "id": text_to_base64(bookname + str(page_num)),
                        "title": f"{bookname}_page_{str(page_num)}",
                        "chunk": content,
                        "chunkVector": embedder.embed_query(content if content!="" else "-------"),
                        "name": bookname,
                        "location": book_url,
                        "page_num": page_num,
                        "sectionheading": sectionheading,
                        "@search.action": "upload"
                    },
                ]
            }

            r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name + "/docs/index",
                                 data=json.dumps(upload_payload), headers=headers, params=params)
            if r.status_code != 200:
                print(r.status_code)
                print(r.text)
        except Exception as e:
            print("Exception:",e)
            #print(content)
            continue

100%|██████████| 38/38 [00:07<00:00,  4.89it/s]


## Query the Index

In [1]:
# QUESTION = "what normally rich dad do that is different from poor dad?"
# QUESTION = "Tell me a summary of the book Boundaries"
# QUESTION = "Dime que significa la radiacion del cuerpo negro"
# QUESTION = "what is the acronym of the main point of Made to Stick book"
QUESTION = "Show all the compliance references for the standart EN 45502"#"Show the file with document number D1330781"
# QUESTION = "who won the soccer worldcup in 1994?" # this question should have no answer

In [20]:
vector_indexes = [book_index_name]

ordered_results = get_search_results(QUESTION, vector_indexes, 
                                        k=10,
                                        reranker_threshold=1,
                                        vector_search=True, 
                                        similarity_k=2, #10,
                                        query_vector = embedder.embed_query(QUESTION)
                                        )

**Note**: that we are picking a larger k=10 since these chunks are NOT of 5000 chars each like prior notebooks, but instead each page is a chunk.

In [21]:
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [22]:
top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location+os.environ['BLOB_SAS_TOKEN']}))
        
print("Number of chunks:",len(top_docs))

Number of chunks: 2


In [23]:
# Calculate number of tokens of our docs
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    prompt_tokens = num_tokens_from_string(COMBINE_PROMPT_TEMPLATE) # this is a custom function we created in common/utils.py
    context_tokens = num_tokens_from_docs(top_docs) # this is a custom function we created in common/utils.py
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

System prompt token count: 1669
Max Completion Token count: 1000
Combined docs (context) token count: 1245
--------
Requested token count: 3914
Token limit for gpt-35-turbo : 4096
Chain Type selected: map_reduce


In [24]:
if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)

In [25]:
%%time
# Try with other language as well
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

CPU times: user 39.1 ms, sys: 605 µs, total: 39.7 ms
Wall time: 6.39 s


In [26]:
display(Markdown(response['output_text']))

There are two compliance references for the standard EN 45502: EN 45502-2-3:2010 Compliance Assessment and EN 45502-1:2015 Clause 5 Compliance Assessment<sup><a href="https://digdaaidasopenaipoc01.blob.core.windows.net/ew-test-custom-skill/./data/books/4 GSPR and Standards/D1336402_9-CI600_EN_45502-2-3_2010_Compliance_Report.pdfsp=rl&st=2023-09-04T11:30:37Z&se=2023-09-28T19:30:37Z&spr=https&sv=2022-11-02&sr=c&sig=m2NcNRPDsm43qrhBkCHD7gIgzOk0v9DIRWRFIBHF4SI%3D">[1]</a></sup>.

# Summary

In this notebook we learned how to deal with complex and large Documents and make them available for Q&A over them using [Hybrid Search](https://learn.microsoft.com/en-us/azure/search/search-get-started-vector#hybrid-search) (text + vector search).

We also learned the power of Azure Document Inteligence API and why it is recommended for production scenarios where manual Document parsing (instead of Azure Search Indexer Document Cracking) is necessary.

Using Azure Cognitive Search with its Vector capabilities and hybrid search features eliminates the need for other vector databases such as Weaviate, Qdrant, Milvus, Pinecone, and so on.


# NEXT
So far we have learned how to use OpenAI vectors and completion APIs in order to get an excelent answer from our documents stored in Azure Cognitive Search. This is the backbone for a GPT Smart Search Engine.

However, we are missing something: **How to have a conversation with this engine?**

On the next Notebook, we are going to understand the concept of **memory**. This is necessary in order to have a chatbot that can establish a conversation with the user. Without memory, there is no real conversation.